# はじめに

このノートではLightGBMのランキング学習を理解するために、末尾に記載の資料を参考にしながら、ランキング学習の基礎からLightGBMのランキング学習アルゴリズムについて理解を深める。ただ、ランキング学習に関して、体系的に学べる書籍やサイトを見つけることができず、様々な資料を参考にしているため、部分的に私の解釈が誤っている可能性がある。本来は下記の書籍を読むのが望ましいと思われるが、高価なため読めていない。

- [Learning to Rank for Information Retrieval | Liu, Tie-Yan ](https://www.amazon.co.jp/Learning-Rank-Information-Retrieval-Tie-Yan/dp/3642142664)



## ランキング学習とは

ランキング学習とは、Learning to Rank(LTR)と呼ばれているもので、回帰や分類などの機械学習モデルを作成する際のデータセットに特徴を持つ。回帰や分類では1レコードが1つのまとまりと解釈できるが、ランキング学習で使用するデータは複数行で1つのまとまりとして解釈し、そのまとまりの中で相対的にランキングを学習することになる。

Learning to Rankは、クエリとの関連性の観点からアイテムのリストを並べ替えることを目的とした教師あり機械学習アルゴリズムとして説明されることが多い。1つのクエリ($q$)に対して、複数のドキュメント($d_1, d_2,...,d_n$)があった場合、どのドキュメントを返すことが望ましいのか。これ機械的に行うためには、モデルはクエリとドキュメントを受け取って、関連性スコアを計算し、それをもとにランキングを返す必要がある。この一連の作業がランキング学習の基本的なふるまいとなるが、どうやって学習するのか、どうやってスコアをつけるのか、それらの考え方の違いにより、いくつかの方法が提案されている。

大まかな時系列としては、ランキング学習の初期アプローチとして、RankNetが存在しており、その発展形でLambdaRankが登場し、ランキングの順位を直接最適化する流れに移り、LambdaRankを拡張したLambdaMartが大規模なランキングタスクに適用されるようになったという流れがありそう。ほかにも様々なアルゴリズムが提案されているが細かいところは専門家ではないので、わからない。

## ランキング学習の方法

ここではPointwise Ranking、Pairwise Rankin、Listwise Rankingの3つランキング学習の方法をまとめておく。特徴量には、ドキュメントから得られる長さ、リンク数や、クエリから得られるクエリの長​​さ、頻度、ドキュメントとクエリの組み合わせから得られるTF-IDF、BM25、BERTなどのスコアが利用される。他にも、Webページから得られるクリック数、滞在時間、コンバージョン数などが利用される。



### Pointwise Ranking

Pointwise Rankingでは、訓練データ内の各クエリとドキュメントのペアが数値または関連性スコアを持っていることを想定する。そして、ランキング学習を回帰の問題として考え、関連性スコアを学習する。下記のようなデータセットのイメージ。

- $q1-d1-score1$
- $q1-d2-score2$
- $q1-d3-score3$

ただ、この学習方法はランキング学習における欠点が指摘されている。上記の例では、$q1$と$d_1, d_2,d_3$は1つのまとまりと解釈できる一方で、Pointwise Rankingはそれを考慮しない。各レコードを独立したデータとして考え、スコアが必要となる。また、各レコードがスコアを持つ必要があるものの、数値はついていても、何に対しての数値なのかがわからない。1つのまとまりの中での100点
は、他のまとまりの中での100点として扱っても問題がないのか。ドキュメントペアのエラーを小さくしようと学習する(選択されたMSEなどの損失関数)。つまり、エラーの最小化とランキングが高いことは同等ではないものの、エラー最小化が目的となるため、ユーザー体験のような観点が考慮されない。Pointwise Rankingはいくつかの欠点があるため、使用する場合は工夫が求められる。



### Pairwise Ranking

Pairwise Rankingでは、訓練データ内の各クエリとドキュメントのまとまりが数値または順序スコアを持っていることを想定する。このアプローチで学習するアルゴリズムとして一般的なものがRankNet、LambdaMARTだと思われる。クエリとドキュメントのペアが与えられた場合、1つまとまりの中で、関連性スコアを計算することでランキングを返す。つまり、クエリに対するドキュメントの相対的な関連性を表すことができる。Pair-input modelsでは、$q1$と$d_1, d_2,d_3$があった場合、下記のようなデータセットのイメージとなる。ドキュメントの順序は$(d_i,d_j)$と表される。$(d_i \succ d_j)$という$d_i$が$d_j$よりもランキングが上位となる確率を計算するために、モデルの重みを勾配降下法によって調整する。

- $q1-(d1,d2)-score1$
- $q1-(d1,d3)-score2$
- $q1-(d2,d3)-score3$

Single-input modelsは単一の特徴量を入力として受け入れ、ペア内の各ドキュメントが独立してモデルに入力され、関連性スコアを受け取る。次に、両方のスコアが比較され、正解ラベルに基づいて勾配降下法によってモデルが調整される。

- $(q,d_i)->f(q,d_i)=s_i$
- $(q,d_j)->f(q,d_j)=s_j$
  - $g(s_i,s_j) = p(d_i \succ d_j)$

一般に、Pairwise Rankingの学習時の損失関数は、定数$\phi$を掛けた2つのスコア$s_i - s_j$の差を引数$z$として受け取り、ロジスティック関数などで確率に変換する。ただ、こちらの学習方法も欠点が指摘されている。データセットが大きくなると、ドキュメントのペアが膨大($n(n-1)/2$)となってしまい、モデルの学習が終わらない可能性がある。また、すべてのドキュメントのペアごとの確率を計算できたとしても、それらを最終的にどのようにランク付けするかは明らかではない。そのため、$(d_1 \succ d_2), (d_2 \succ d_3), (d_3 \succ d_1)$のような逆説的な関係が生まれる可能性がある。



### Listwise Ranking

Listwise Rankingでは、訓練データ内のすべてのクエリの評価尺度の1つの値を直接最適化する。ランキングモデルのパラメタに関して、評価尺度は連続関数ではないため、評価尺度の近似を行う。リスト内の項目間の関係を学習したモデルを使用して、特徴量に基づいて新しいドキュメントに関連性スコアを割り当てることができ、スコアの降順に並び替えることでランキングを得ることができる。


## RankNet,  LambdaRank, LambdaMart

ここではRankNet,  LambdaRank, LambdaMartについて理解を深める。






### RankNet

ペアワイズ損失は、ランキングにおける平均反転回数(inversions in ranking)を最小化することを目指す。$d_i \succ_q d_j$ではあるものの$d_j$は$d_i$よりも高くランキングされる可能性がある。

$$
\mathcal{L} = \phi(s_i - s_j)
$$

$\phi$は$\phi(z)=log(1+e^{-z})$が一般的に使用される。他にも$\phi(z)=max(0,1-z)$や$\phi(z)=e^{-z}$が利用される。RankNetでは、

$$
P_{ij} = P(s_i > s_j) \equiv \frac{e^{\sigma s_i}}{e^{\sigma s_i}+e^{\sigma s_j}} = \frac{1}{1+e^{-\sigma(s_i - s_j)}} 
$$

として確率を予測する。$P_{ji} \equiv \frac{1}{1+e^{-\sigma(s_j - s_i)}}$である。ここでどのような確率が望ましいかを考えると、$\bar{P}_{ij}=1,\bar{P}_{ji}=0$である。$\bar{P},P$の関係について、クロスエントロピーを計算できる。ここでは対数尤度からクロスエントロピーに変換している。

$$
\begin{align*}
\mathcal{LL} &= \bar{P}_{ij} log(P_{ij}) + (1-\bar{P}_{ij})log(P_{ij}) \\
&= \bar{P}_{ij}\log(\frac{1}{1 + e^{-\sigma (s_{i} - s_{j})}}) + (1 - \bar{P}_{ij})\log(\frac{e^{-\sigma(s_{i} - s_{j})}}{1 + e^{-\sigma(s_{i} - s_{j})}}) \\
&= -\bar{P}_{ij}\log(1 + e^{-\sigma (s_{i} - s_{j})}) + \log(e^{-\sigma (s_{i} - s_{j})}) - \log(1 + e^{-\sigma (s_{i} - s_{j})}) - \bar{P}_{ij}\log(e^{-\sigma (s_{i} - s_{j})}) + \bar{P}_{ij}\log(1 + e^{-\sigma (s_{i} - s_{j})}) \\
&= (1 - \bar{P}_{ij})\log(e^{-\sigma (s_{i} - s_{j})}) - \bar{P}_{ij}\log(1 + e^{-\sigma (s_{i} - s_{j})}) \\
\end{align*}
$$

対数尤度から考えているので、尤度にマイナスをつけ、クロスエントロピーを考える。つまりクロスエントロピーを最小化する。

$$
\mathcal{L} = (\bar{P}_{ij}-1)\log(e^{-\sigma (s_{i} - s_{j})}) + \bar{P}_{ij}\log(1 + e^{-\sigma (s_{i} - s_{j})})
$$

$\bar{P}_{ij}$があればよいので、$\bar{P}_{ij}=1$を考えると、

$$
\mathcal{L} = (1-1)\log(e^{-\sigma (s_{i} - s_{j})}) + 1\log(1 + e^{-\sigma (s_{i} - s_{j})}) = \log(1 + e^{-\sigma (s_{i} - s_{j})})
$$

であり、$\bar{P}_{ij}=0$を考えると、

$$
\mathcal{L} = (1-0)\log(e^{-\sigma (s_{i} - s_{j})}) + 0\log(1 + e^{-\sigma (s_{i} - s_{j})}) = \log(1 + e^{-\sigma (s_{i} - s_{j})})
$$

となり、シンプルに考えることができる。ここで、RankNetの有名な因数分解(Factorization)を利用する。$S_{ij} \in \{-1,0,1\}$が$d_i$と$d_j$の間の選好を示すとすると、あるペアに対する望ましい確率は次のようになる。

$$
\bar{P}(d_i \succ d_j) = \frac{1}{2}(1+S_{ij}) 
$$

$S=-1, \frac{1}{2}(1+(-1))=0$、$S=0, \frac{1}{2}(1+(0))=0.5$、$S=1, \frac{1}{2}(1+(1))=1$となる。そして、予測される確率は次の通り。

$$
P(d_i \succ d_j) = \frac{1}{1+e^{-\sigma(s_i - s_j)}}
$$

クロスエントロピー損失は次の通り。

$$
\mathcal{L} = \frac{1}{2}(1-S_{ij})\sigma(s_i - s_j) + log(1+ e^{-\sigma(s_i - s_j)})
$$

これは頑張ってゴリゴリ計算すれば導出できる。

<img src='./cross-entropy1.png'>

クロスエントロピー損失を$s_i,s_j$に関して微分すると

$$
\begin{align*}
\frac{\partial \mathcal{L}_{ij} }{\partial s_i} &= \sigma \left( \frac{1}{2} (1 - S_{ij}) - \frac{1}{1+e^{-\sigma(s_i - s_j)}} \right) \\
\frac{\partial \mathcal{L}_{ij} }{\partial s_j} &= -\sigma \left( \frac{1}{2} (1 - S_{ij}) - \frac{1}{1+e^{-\sigma(s_i - s_j)}} \right)
\end{align*}
$$

となる。そして、これをさらに分解するとこうなる。

$$
\begin{align*}
\frac{\partial \mathcal{L}_{ij} }{\partial w} &= \frac{\partial \mathcal{L}_{ij} }{\partial s_i}\frac{\partial s_{i} }{\partial w} + \frac{\partial \mathcal{L}_{ij} }{\partial s_j}\frac{\partial s_{j} }{\partial w} \\
&= \sigma \left( \frac{1}{2} (1 - S_{ij}) - \frac{1}{1+e^{-\sigma(s_i - s_j)}} \right) \left( \frac{\partial s_i}{\partial w} - \frac{\partial s_j}{\partial w}\right)
\end{align*}
$$

$s_i,s_j$は何らかの重み$w$を最適化するモデルの関数から得られ、上記の勾配はその$s$の関数である。ここで、$\mathcal{L}{ij}$は$s_i$および$s_j$の関数であり、$s_i$と$s_j$はそれぞれ重み$w$に関するモデルの出力。したがって、$\frac{\partial \mathcal{L}{ij}}{\partial w}$を計算するためには、$\frac{\partial \mathcal{L}{ij}}{\partial s_i}$および$\frac{\partial \mathcal{L}{ij}}{\partial s_j}$を$w$に関して微分する必要がある。

$\frac{\partial \mathcal{L}_{ij}}{\partial s_i}$と$\frac{\partial \mathcal{L}{ij}}{\partial s_j}$は既に与えられており、これらはそれぞれ$s_i$と$s_j$の関数です。次に、$s_i$と$s_j$が$w$の関数であることを考慮すると、連鎖率を使って$\frac{\partial s_i}{\partial w}$および$\frac{\partial s_j}{\partial w}$を計算することができる。そして、これらの微分を$\frac{\partial \mathcal{L}{ij}}{\partial s_i}$と$\frac{\partial \mathcal{L}{ij}}{\partial s_j}$に乗じて合計することで、$\frac{\partial \mathcal{L}_{ij}}{\partial w}$を得られる。連鎖率を適用し、微分の和として表させる。

ここで、$\lambda_{ij} = \sigma \left( \frac{1}{2} (1 - S_{ij}) - \frac{1}{1+e^{-\sigma(s_i - s_j)}} \right)$とおくと、次のように書ける。$\lambda_{ij}$は、ドキュメントのペアを近づけたり、遠ざけるような働きをする。$\lambda_{i} = \sigma_{j} \lambda_{ij}$である。

$$
\frac{\partial \mathcal{L}_{ij}}{\partial w} = \lambda_{ij} \left( \frac{\partial s_i}{\partial w} - \frac{\partial s_j}{\partial w}\right)
$$

このモデルは、ドキュメントを独立してスコア化$f(\vec{x}_{d_{i}})=s_i$する。この損失はドキュメントのペアに基づいており、不正確な反転数を最小化する。

$$
\mathcal{L}_{pairwise} = \sum_{d_i \succ d_j} \phi(s_i - s_j) 
$$

RankNetであれば下記を最小化する。

$$
\mathcal{L}_{RankNet} = \sum_{d_i \succ d_j} log(1 + e^{-\sigma(s_i - s_j)}) 
$$

不正確な反点数を減少させることが、必ずしもランキング観点で望ましいとは言えない点がモデルの欠点と指摘される。モデルが出力する確率は、特定のドキュメントが上位にランキングされるという確信度合いを示す。ただ、前に見た悪循環を伴って、混乱を招くケースが起こり得る。また、すべてのドキュメントのペアが同じように重要ではない。上位のドキュメントの順序を正しくすることは、下位のドキュメントの順序を正しくするよりもはるかに重要である。例えば、上位5位までのドキュメントの順番は、10位以降のドキュメントの順番よりもはるかに重要という意味である。

下記のケース1では、pos11をpos2の位置に持ってくるためには、反点数として9回必要になる。

- pos1 - d_1 - Relevant
- pos2 - d_2 - NotRelevant
- ...
- pos10 - d_10 - NotRelevant
- pos11 - d_11 - Relevant

下記のケース2では、pos5,6をpos1,2の位置に持ってくるためには、反点数として8回必要になる。

- pos1 - d_1 - NotRelevant
- pos2 - d_2 - NotRelevant
- ...
- pos5 - d_5 - Relevant
- pos6 - d_6 - Relevant
- ...
- pos10 - d_10 - NotRelevant
- pos11 - d_11 - NotRelevant

アルゴリズムとしては、ケース2のほうが反点数が少なく、優秀と考えることができるが、ユーザー観点では関連するものが最も上に位置しているケース1のほうが望ましい。結果として、すべてのドキュメントのペアが同じように重要であるわけではないため、上位を正しくランキングできることをより重視するアルゴリズム(nDCGを評価指標して)が求められる。

### LambdaRank, LambdaMart

RankNetの問題点を修正しているがLambdaRank。Listwise Rankingアプローチの考え方では、訓練データ内のすべてのクエリの評価尺度の1つの値(DCG)を直接最適化する。つまり、DCGのようなメトリクスを使用して、ランキングの質を直接的に最適化することを目指す。DCGについては、この次の章でまとめるが、ここではnDCGの値が大きいほど、ランキングの品質が高くなる指標と考えておけば良い。

$$
\begin{align*}
DCG &= r_1 + \sum_{i=2}^{n} \frac{r_i}{log_2 i} \\
nDCG &= \frac{DCG_{pred}}{DCG_{true}} 
\end{align*}
$$

ただ、ここで1つの問題が発生する。DCGのようなメトリクスを最適化すれば良さそうというのはわかるが、どうやって最適化すればよいのだろうか。DCGのようなメトリクスは非連続で、微分不可能である。

$$
\frac{\partial DCG}{\partial w} = \frac{\partial r_1 + \sum_{i=2}^{n} \frac{r_i}{log_2 i}}{\partial w} = ?
$$

ここでモデルを振り返って必要なものが何かを考える。モデルを訓練するために必要なのは、コストではなく、モデルのスコアに対するコストの勾配であり、勾配は、位置の入れ替えによってnDCGに大きな影響を与えるドキュメントのペアほど大きくなるはずである。このような考察から、Burges(2006)がRankNetの損失値に|nDCG|を掛けた場合、アルゴリズムは直接nDCGを最適化する傾向があることを証明した。

- [C. J. Burges, R. Ragno, and Q. V. Le. Learning to rank with nonsmooth cost functions. In NIPS,volume 6, pages 193–200, 2006.](https://papers.nips.cc/paper_files/paper/2006/hash/af44c4c56f385c43f2529f9b1b018f6a-Abstract.html)

下記、論文のアブストラクトの引用である。

> 情報検索で使われる品質尺度は、特に直接最適化するのが難しい。なぜなら、与えられたクエリに対して返される文書のソート順を通してのみ、モデルスコアに依存するからである。したがって、モデルパラメータに関するコストの導関数はゼロか、あるいは未定義である。本稿では、LambdaRankと呼ばれるシンプルで柔軟なアルゴリズムを提案する。LambdaRankをニューラルネットワークモデルを用いて説明するが、この考え方はどのような微分可能関数クラスにも適用可能である。結果として得られる陰的コスト関数が凸であるための必要十分条件を与え、一般的な手法が単純な力学的解釈を持つことを示す。いくつかのデータセットにおいて、最新のランキングアルゴリズムよりも有意に精度が向上したことを示す。また、LambdaRankがランキングアルゴリズムの学習段階を有意に高速化する手法を提供することも示す。本論文はランキングを対象としているが、提案手法はあらゆる非平滑・多変量コスト関数に拡張可能である。

これはnDCGだけでなく、他のメトリクスでも同様に機能する。これがLambdaRankと呼ばれるものである。LambdaRankがRankNetよりも優れたパフォーマンスを発揮できる。$\Delta nDCG$はランキングを変更した際のnDCGの変化を示しており、$\lambda_{RankNet}$は通常のRankNetにおける学習率。LambdaRankでは、$\lambda_{LambdaRank}$を$\lambda_{RankNet}$に対して$nDCG$の変化に比例させることで、ランキングの質を改善する方向に対してより大きな学習率を適用する。これにより、ランキングの品質を直接的に最適化することが可能となる。

$$
\lambda_{LambdaRank} = \lambda_{RankNet} |\Delta nDCG|
$$

LambdaRankではRankNetと同様にペアワイズのランキングを学習する。RankNetで使用される損失関数は微分可能であり、モデルの重み$w$に対する勾配を計算できる。そのため、損失関数を最小化する際には、通常の勾配降下法やその他の最適化アルゴリズムを使用できる。一方、ランキングの品質を直接最適化するために使用されるnDCGは、非連続で微分不可能。そのため、これらのメトリクスを直接最適化することは困難ではあるが、LambdaRankでは、nDCGの変化に基づいて重み$w$を更新することで、ランキングの品質を改善しようと試みる。このようにして微分不可能な問題を回避し、DCGを最適化するために、RankNetを利用しながら学習する。さらに発展させたものとして、LambdaMartがある。LambdaMartは、LambdaRankを勾配ブースティングで学習することで、LambdaRankよりも優れたパフォーマンスを期待できる。

LambdaRankの中身はPairwiseアプローチであるRankNetを利用していることからも分かる通り、LambdaRankはListwise Rankingアプローチと呼ばれるかもしれないが、Pairwise、Listwiseの区別はあまり重要ではない。どちらかといえば、「Pointwise Ranking」と「Pairwise Rankin、Listwise Ranking」の区別のほうが重要かもしれない。

## ランキングの評価指標

ランキングの評価指標には、Precision、Recall、top@k、平均適合率(AP)、逆数順位(reciprocal rank)、幾何平均適合率(geometric mean average precision)、ランク有効性指標(rank effectiveness measure)などがあるが、ここではnDCGに焦点をあてる。他の指標に関しては検索すれば多くのわかりやすい資料がでてくるので、そちらを参照。

### nDCG(normalized Discounted Cumulative Gain)

DCGの定義には2種類ある。ここでは1つ目の方のDCGを利用するnDCGについてまとめる。DCGの特徴は、ランキング上位での正解はDCGへの寄与が大きく、ランキング下位で正解してもDCGへの寄与は小さくなる。

$$
\begin{align*}
DCG &= r_1 + \sum_{i=2}^{n} \frac{r_i}{log_2 i} \\
DCG &= \sum_{i=2}^{n} \frac{2^{r_i}-1}{log_2 (i+1)}
\end{align*}
$$

nDCGは0から1の値を取り、1に近いほど正しいランキング予測結果であることを意味する。nDCGは、予測されたDCGと正解のDCGで割ることで正規化される。

$$
\begin{align*}
nDCG &= \frac{DCG_{pred}}{DCG_{true}} 
\end{align*}
$$

こちらのブログにnDCGのPythonでの実装が書かれているので、参考にさせていただく。

- [検索の評価に関するあれこれ - Re:ゼロから始めるML生活](https://www.nogawanogawa.com/entry/ndcg#%E5%8F%A4%E5%85%B8%E7%9A%84%E3%81%AA%E8%A9%95%E4%BE%A1%E6%8C%87%E6%A8%99)

他にも、下記の方のコードも参考になる。

- [Learning to rank metrics.](https://gist.github.com/mblondel/7337391)

In [3]:
import numpy as np
from sklearn.metrics import ndcg_score

def dcg(gain, k=None):
    """ calc dcg value """ 
    if k is None:
        k = gain.shape[0]

    ret = gain[0]
    for i in range(1, k):
        ret += gain[i] / np.log2(i + 1)
    return ret


def ndcg(y, k=None, powered=False) -> float:
    """ calc nDCG value """

    dcg_score = dcg(y, k=k)

    ideal_sorted_scores = np.sort(y)[::-1]
    ideal_dcg_score = dcg(ideal_sorted_scores, k=k)

    print("pred_ranking : {}".format(y))
    print("ideal_ranking : {}".format(ideal_sorted_scores))

    print("pred_sorted_scores : {:.2f}".format(dcg_score))
    print("ideal_dcg_score : {:.2f}".format(ideal_dcg_score))

    return print("nDCG : {:.2f}".format(dcg_score / ideal_dcg_score))

例えば、`[0,2,0,1,0]`という予測を得た時、nDCGは下記の通り計算される。

$$
\begin{align*}
DCG_{pred} &= 0 + \frac{2}{log_{2}2} + \frac{0}{log_{2}3} + \frac{1}{log_{2}4}+ \frac{0}{log_{2}5} = 2.5 \\
DCG_{ideal} &= 2 + \frac{1}{log_{2}2} + \frac{0}{log_{2}3} + \frac{0}{log_{2}4}+ \frac{0}{log_{2}5} = 3.0 \\
nDCG &= \frac{DCG_{pred}}{DCG_{ideal}} = \frac{2.5}{3.0} = 0.833
\end{align*}
$$

In [4]:
ndcg(np.array([0,2,0,1,0]))

pred_ranking : [0 2 0 1 0]
ideal_ranking : [2 1 0 0 0]
pred_sorted_scores : 2.50
ideal_dcg_score : 3.00
nDCG : 0.83


### ndcg_score関数

`sklearn`ライブラリの`metrics`モジュールには`ndcg_score`関数が用意されており、この関数を使用することでnDCGを計算できる。引数は正解のランキングと関連性スコアの予測値で、予測値をランキングに変換することなく、指標を計算してくれる。

In [36]:
y_true = np.array([4, 3, 2, 1, 0])  
y_score = np.array([0.4, 0.1, 0.0, 0.2, 0.3])  

ndcg_score([y_true], [y_score])

0.8965338886194285

「scikit-learn/sklearn/metrics/_ranking.py」の実装も確認しておく。下記の公開されているコードを読んだところ、`_dcg_sample_scores`関数がそのあたりを計算してくれていることがわかる。

- [Learning to rank metrics.](https://github.com/scikit-learn/scikit-learn/blob/8721245511de2f225ff5f9aa5f5fadce663cd4a3/sklearn/metrics/_ranking.py#L1431C1-L1488C28)

そのため、関数に渡す前にスコアをランキングに変換したものを渡しても、同様に計算される。

In [37]:
y_score = np.array([4, 1, 0, 2, 3])  

ndcg_score([y_true], [y_score])

0.8965338886194285

## 参考資料

- [Learn the Basic Steps for Implementing Learning to Rank Techniques](https://towardsdatascience.com/what-is-learning-to-rank-a-beginners-guide-to-learning-to-rank-methods-23bbb99ef38c)
- [Introduction to Ranking Algorithms](https://towardsdatascience.com/introduction-to-ranking-algorithms-4e4639d65b8)
- [IR-2021-5-offline-LTR-handout.pdf](https://drive.google.com/file/d/1T4PGLQbAxUgRxtY5KVSWYC4qz4rCh86X/view?usp=drive_link)
- [Ranknet loss derivation](https://medium.com/@lautaro.quiroz/ranknet-loss-derivation-1b62c33bae48)
- [Understand and Implement Evaluation Metrics for Learning to Rank Techniques | Towards Data Science](https://towardsdatascience.com/how-to-evaluate-learning-to-rank-models-d12cadb99d47)
- [予測ランキング評価指標：NDCGの2つの定義と特徴の比較 - 人間だったら考えて](https://www.szdrblog.info/entry/2017/02/24/235539)
- [検索の評価に関するあれこれ - Re:ゼロから始めるML生活](https://www.nogawanogawa.com/entry/ndcg)
- [レコメンドつれづれ　～第3回 レコメンド精度の評価方法を学ぶ～ | DOORS DX](https://www.brainpad.co.jp/doors/contents/01_tech_2017-08-25-140000/)
